In [ ]:
!pip install ultralytics -q
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")

input_video = "video.mp4"
output_video = "output_detected.mp4"

cap = cv2.VideoCapture(input_video)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, imgsz=640, conf=0.25, verbose=False)
    for result in results:
        annotated_frame = result.plot()

    out.write(annotated_frame)

cap.release()
out.release()

In [ ]:
import cv2
import torch
import time
from pathlib import Path
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

video_path = '/content/video.mp4'

model_performance = {}

for model_name in ["yolov8n", "yolo11n", "yolo12n"]:
    try:
        model = YOLO(f"{model_name}.pt")
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            continue

        start_time = time.time()
        frames_processed = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            results = model(frame, verbose=False)
            frames_processed += 1

        end_time = time.time()
        cap.release()

        inference_time = end_time - start_time
        calculated_fps = frames_processed / inference_time if inference_time > 0 else 0

        model_performance[model_name] = {
            'inference_time_seconds': inference_time,
            'processed_frames': frames_processed,
            'calculated_fps': calculated_fps
        }
        print(f"Finished processing with {model_name}")

    except Exception as e:
        print(f"Could not process with model {model_name}: {e}")
        model_performance[model_name] = {'error': str(e)}


print("\n--- Model Performance Comparison ---")
for model_name, metrics in model_performance.items():
    print(f"\nModel: {model_name}")
    if 'error' in metrics:
        print(f"  Status: Error - {metrics['error']}")
    else:
        print(f"  Total Inference Time: {metrics['inference_time_seconds']:.2f} seconds")
        print(f"  Frames Processed: {metrics['processed_frames']}")
        print(f"  Calculated FPS: {metrics['calculated_fps']:.2f}")

print("------------------------------------")


Finished processing with yolov8n
Finished processing with yolo11n
Finished processing with yolo12n

--- Model Performance Comparison ---

Model: yolov8n
  Total Inference Time: 91.54 seconds
  Frames Processed: 472
  Calculated FPS: 5.16

Model: yolo11n
  Total Inference Time: 72.00 seconds
  Frames Processed: 472
  Calculated FPS: 6.56

Model: yolo12n
  Total Inference Time: 91.67 seconds
  Frames Processed: 472
  Calculated FPS: 5.15
------------------------------------


In [ ]:
model_detailed_object_counts = {}

for model_name in ["yolov8n", "yolo11n", "yolo12n"]:
    try:
        model = YOLO(f"{model_name}.pt")
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            model_detailed_object_counts[model_name] = {'error': f"Could not open video file {video_path}"}
            continue

        total_objects_detected = 0
        frames_processed = 0
        object_class_counts = {}

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, verbose=False)
            for result in results:
                if result.boxes is not None:
                    total_objects_detected += len(result.boxes)
                    for box in result.boxes:
                        class_id = int(box.cls[0])
                        class_name = model.names[class_id]
                        object_class_counts[class_name] = object_class_counts.get(class_name, 0) + 1


            frames_processed += 1

        cap.release()

        model_detailed_object_counts[model_name] = {
            'total_objects_detected_across_video': total_objects_detected,
            'frames_processed': frames_processed,
            'object_class_counts': object_class_counts
        }
        print(f"Finished detailed object counting for {model_name}")

    except Exception as e:
        print(f"Could not process with model {model_name}: {e}")
        model_detailed_object_counts[model_name] = {'error': str(e)}

print("\n--- Model Detailed Object Detection Comparison ---")
for model_name, metrics in model_detailed_object_counts.items():
    print(f"\nModel: {model_name}")
    if 'error' in metrics:
        print(f"  Status: Error - {metrics['error']}")
    else:
        print(f"  Total Objects Detected Across All Processed Frames: {metrics['total_objects_detected_across_video']}")
        print(f"  Frames Processed: {metrics['frames_processed']}")
        print("  Detected Objects and Counts:")
        if metrics['object_class_counts']:
            for class_name, count in metrics['object_class_counts'].items():
                print(f"    - {class_name}: {count}")
        else:
            print("    No objects detected.")


print("-------------------------------------------------")

Finished detailed object counting for yolov8n
Finished detailed object counting for yolo11n
Finished detailed object counting for yolo12n

--- Model Detailed Object Detection Comparison ---

Model: yolov8n
  Total Objects Detected Across All Processed Frames: 1814
  Frames Processed: 472
  Detected Objects and Counts:
    - cow: 206
    - horse: 406
    - giraffe: 122
    - elephant: 398
    - person: 25
    - bird: 480
    - zebra: 165
    - sheep: 11
    - bear: 1

Model: yolo11n
  Total Objects Detected Across All Processed Frames: 1575
  Frames Processed: 472
  Detected Objects and Counts:
    - cow: 312
    - horse: 422
    - elephant: 428
    - giraffe: 113
    - person: 12
    - sheep: 1
    - bird: 284
    - zebra: 3

Model: yolo12n
  Total Objects Detected Across All Processed Frames: 1687
  Frames Processed: 472
  Detected Objects and Counts:
    - cow: 45
    - horse: 773
    - person: 9
    - giraffe: 113
    - elephant: 311
    - bird: 436
---------------------------------